In [186]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

In [187]:
def fetch_data():
    file_path = "company_esg_score.xlsx"
    df = pd.read_excel(file_path, header =0)
    return df

In [188]:
df = fetch_data()

In [189]:
app = Dash(__name__)

In [190]:
total_score = df['Total Score'].iloc[-1]

In [191]:
def determine_esg_level(total_score):
    if total_score >= 85.72:
        return "AAA"
    elif total_score >= 71.44:
        return "AA"
    elif total_score >= 57.15:
        return "A"
    elif total_score >= 42.87:
        return "BBB"
    elif total_score >= 28.68:
        return "BB"
    elif total_score >= 14.30:
        return "B"
    else:
        return "CCC"

In [192]:
def create_gauge(total_score):
    fig_gauge = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = total_score,
        title = {'text': "ESG Score"},
        gauge = {
            'axis': {'range': [0, 100]},
            'steps': [
                {'range': [0, 40], 'color': "white"},
                {'range': [40, 60], 'color': "lightgreen"},
                {'range': [60, 80], 'color': "limegreen"},
                {'range': [80, 100], 'color': "green"},
            ],
        },
    ))
    return fig_gauge


In [193]:
@app.callback(
    [Output('gauge-chart', 'figure'),
     Output('esg-level-text', 'children')],  
    Input('dummy-input', 'value')
)
def update_charts(dummy_value):
    df = fetch_data()
    total_score = df['Total Score'].iloc[-1]
    
    esg_level = determine_esg_level(total_score)
    
    gauge_fig = create_gauge(total_score)
    
    return gauge_fig, f"Industry rating: {esg_level}"  

app.layout = html.Div([
    dcc.Graph(id = 'gauge-chart'),
    dcc.Input(id = 'dummy-input', type = 'hidden', value = 'dummy'),
    html.Div([
        html.P(id = 'esg-level-text', style = {
            'fontSize': '24px',  
            'textAlign': 'center', 
            'color': 'green',
            'position': 'relative',  
            'top': '-30px',
            'lineHeight': '1',    
            'marginTop': '0',     
            'marginBottom': '0',  
            'paddingTop': '0'     
        })
    ])
])

In [194]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 8073)